## Machine Learning and Artificial Intelligence 
Summer High School Academic Program for Engineers (2025)
## Backpropagation

In this notebook, we will learn how the weights of a neural network are optimized. 

### Losses and Error Functions 

As we discussed before, the goal of supervised machine learning is to find a hypothesis function $y=h(x)$ (the model) that approximates the real relation
between the input and the output. We can measure the error the current model makes on the training data. Then, we update the model parameters to minimize this error. 

A **loss function** $\text{loss}_h(y, h(x_i))$ measures the error the model $h$ makes on a single training example $x_i$.

The **empirical error** of the model is the sum or average of the losses for each training data point: $E_{train}(h) = \frac{1}{n} \sum_{i=1}^n \text{loss}(y, h(x_i))$.
Note that the empirical error is a function of the model, that is the learned parameters (weights) of the model.

The **objective** of the machine learning algorithm is to minimize the empirical error. We hope that by doing so we will also minimize the error on the test data $E_{test}$, that is our model generalizes well to unseen data. 

For regression tasks, a common loss is **squared loss**: $\text{loss}(y, h(x_i)) = (y - h(x_i))^2$. In this case the error is called the **mean squared error**: $\frac{1}{n} \sum_{i=1}^n \text{loss}(y, h(x_i)) = \frac{1}{n} \sum_{i=1}^n (y - h(x_i))^2$


For classification tasks, a simple loss function is the **classification loss**, which we implicitly used during perceptron learning. 

$
\text{loss}(y, h(x_i)) =
\begin{cases}
0 & \text{if } \hat{y} = h(x_i) \\
1 & \text{if } \hat{y} \ne h(x_i)
\end{cases}$

Unfortunately, the classification loss is not differentiable, so we cannot use it when training a neural network. 

A better option is to use **negative log likelihood (also known as cross-entropy loss)**. Assume, we have a neural network with $d$ different outputs (a $d$ dimensional output vector) and we use the softmax activation function. The output might look something like this: 

$h(x_i) = \begin{bmatrix}0.35\\0.16\\0.28\\0.21 \end{bmatrix}$

The target vector $y$ is a **1-hot vector** and might look something like this: 

$y = \begin{bmatrix}1\\0\\0\\0 \end{bmatrix}$

Then the **negative log likelihood loss** is 

$- \sum_{i=1}^d y_i \log h(x_i)$

And since all elements of $y$ except one are 0, this is simply $-\log P(c)$ where $c$ is the class of the target class. In this case: $- \log 0.35 \approx 1.0498$ (assuming base $e$, which is typical -- but the choice of the base doesn't affect the learning behavior). 

If the output is not a distribution, but a single sigmoid activated unit (and the target is either $0$ or $1$), the **binary cross-entropy loss** is 
$- [(y \log y') + (1-y) \log (1-y')]$

Recall that the sigmoid function computes the probability for the prediction to be 1. 

### Logistic Regression and Stochastic Gradient Descent

Now consider a single sigmoid actived unit (i.e. a linear model with sigmoid activation instead of the step function used with the perceptron) with binary cross-entropy loss. This is known as a **logistic regression** model. 

<img src="https://www.cs.columbia.edu/~bauer/shape/log_regression_cg.png" width=700px>

We start with some arbitrary weight vector $\mathbf{w}$. We present an input vector $\mathbf{x}$ to the model. During the **forward pass**, we compute the **activation** at each node in the computation graph and the loss. Think of the loss as a function of the weights, $Loss(\mathbf{w})$.

Now, our goal is to update the weight vector $\mathbf{w}$ so as to minimize the loss. We need to compute the **gradient of the loss function with respect to the weights**, that is $\nabla_{\mathbf{w}} Loss(\mathbf{w})$  (also written $\frac{\partial Loss}{\partial \mathbf{w}}$). Assume for now that we have this gradient. Then we shift the weights $\mathbf{w}$ into the opposite direction of the gradient (since we want to minimize the loss and the gradient points in the direction of the steepest increase). 

The update rule becomes: $\mathbf{w} \leftarrow \mathbf{w} - \eta \nabla_{\mathbf{w}} Loss(\mathbf{w})$, 

where $\eta$ is the learning rate. Take a moment to compare this to the perceptron update rule. 

This algorithm is known as **Stochastic Gradient Descent (SGD)**.

**Computing the Gradient:**

We want to compute the gradient $\nabla_{\mathbf{w}} Loss(\mathbf{w}) = \frac{\partial Loss}{\partial \mathbf{w}}$.

Using the **chain rule**, we can decompose this into $\frac{d Loss}{d y'} \frac{\partial y'}{\partial \mathbf{w}} = 
\frac{d Loss}{d y'} \frac{d y'}{d z} \frac{\partial z}{\partial \mathbf{w}}$ 

The derivative of the loss function is 

$\frac{d Loss}{d y'} = \frac{d }{d y'}- [(y \log y') + (1-y) \log (1-y')]$
$ = \frac{y'-y}{y' (1-y')}$

The derivative of the sigmoid is (elegantly)

$\frac{dy'}{dz} = \frac{d}{dz} \sigma(z) = \sigma(z) \cdot (1 - \sigma(z)) = y'  (1-y')$

So the first two factors become: 
$\frac{d Loss}{d y'} \frac{d y'}{d z}  = \frac{y'-y}{y' (1-y')} \cdot y'  (1-y') = y'-y$  
(!!)

For the third factor: 
$\frac{\partial z}{\partial \mathbf{w}} = \frac{\partial \sum_{i} w_i x_i}{\partial \mathbf{w}} = \mathbf{x}$ (consider a single $w_i$: all other $w_j x_j$ terms become 0, and $\frac{\partial w_i x_i}{\partial w_i} = x_i$)

Putting it all together: $\frac{\partial Loss}{\partial \mathbf{w}} = (y'-y) \mathbf{x}$

So the update rule becomes: 
$\mathbf{w} \leftarrow \mathbf{w} - \eta (y'-y) \mathbf{x}$


(Notes: 1. We did not show how the individual derivatives are calculated -- feel free to check. 2. I did not include a bias weight $b$. Remember that the bias weight can be treated as an additional element of $\mathbf{w}$ with constant input 1.)

### Stochastic Gradient Descent on Neural Networks 

We will now apply stochastic gradient descent to multi-layer neural networks. Compared to the logistic regression model, there are two complications. First, we need to propagate the gradient back across multiple layers (using the chain rule). Second, there are many units in each layer, not just one. 

<img src="https://www.cs.columbia.edu/~bauer/shape/nn_flat_cg.png" width=600px>

Recall that each layer $i$ computes an activation (a vector)  $\mathbf{a_i} = f(\mathbf{z_i}) = \mathbf{W_i} \mathbf{a}_{i-1}+\mathbf{b}_i$. Where $f$ is a nonlinear activation function (such as the sigmoid function).
$\mathbf{a}_{i-1}$ is the activation at the previous layer. 

We will use stochastic gradient descent. For a single input vector $\mathbf{x}$, perform a **forward pass** to compute the **forward activations** $\mathbf{a_i}$, then compute the loss. 

Next, we need to calculate the partial derivatives with respect to each weight $\frac{\partial loss}{\partial \mathbf{W}_i}$ and apply the update rule to shift the weight in the opposite direction of the gradient. 

$\mathbf{W}_i \leftarrow \mathbf{W_i} - \eta \frac{\partial loss}{\partial \mathbf{W}_i}$

We iterate over the training data, performing an update after each training example. We repeat this for a pre-defined number of epochs. 

### Backpropagation

But how do we compute $\frac{\partial loss}{\partial \mathbf{W}_i}$?

Unfortunately, we cannot find a closed-form expression for each $\frac{\partial loss}{\partial \mathbf{W}_i}$. Instead, we find the value of the gradient at the current weights algorithmically, by employing the chain rule. The core idea behind the backpropagation algorithm is that we calculate the gradients in a backwards pass, starting at the loss, similar to how the activations are computed in the forward direction. 

Assume we have $\frac{\partial loss}{\partial \mathbf{a}_i}$. Then we compute 

$\delta_i = \frac{\partial loss}{\partial \mathbf{z_i}} = \frac{\partial loss}{\partial \mathbf{a}_i} \frac{\partial \mathbf{a}_i}{\partial \mathbf{z}_i} = \frac{\partial loss}{\partial \mathbf{a}_i} \circ \frac{df}{d \mathbf{z}_i} $ 

Here $\circ$ denotes element-wise multiplication between two vectors because the activation function $f$ is applied element-wise.

We can now use $\delta_i$ in two ways. 

1. to get the loss gradient with respect the weights:
> $\frac{\partial loss}{\partial \mathbf{W}_i} = \frac{\partial loss}{\partial \mathbf{z_i}} \frac{\partial \mathbf{z_i}}{\partial \mathbf{W}_i} = \delta_i \frac{\partial \mathbf{z_i}}{\partial \mathbf{W}_i}$
>
> Since $\mathbf{z}_i = \mathbf{W}_i \mathbf{a}_{i-1} + \mathbf{b}$, we get $\frac{\partial \mathbf{z_i}}{\partial \mathbf{W}_i} = \mathbf{a}_{i-1}^\intercal$
>
> So: $\frac{\partial loss}{\partial \mathbf{W}_i} = \delta_i \mathbf{a}_{i-1}^\intercal$
>
> (To see why we need to take the transpose: If $\delta_i$ is a column vector of size $d \times 1$ and $\mathbf{a}_{i-1}^\intercal$ is $1 \times e$, the result is a $d \times e$ matrix—just like $\mathbf{W}_i$.)

2. to pass the gradient to the previous layer:
> $\frac{\partial loss}{\partial \mathbf{a}_{i-1}} = \frac{\partial loss}{\partial \mathbf{z_i}} \frac{\partial \mathbf{z_i}}{\partial \mathbf{a}_{i-1}} =  \delta_i \frac{\partial \mathbf{z_i}}{\partial \mathbf{a}_{i-1}}$
> 
> And again because $\mathbf{z}_i = \mathbf{W}_i \mathbf{a}_{i-1} + \mathbf{b}$, we get
> $\frac{\partial \mathbf{z_i}}{\partial \mathbf{a}_{i-1}} = \mathbf{W}_i^\intercal$
>
> So $\frac{\partial loss}{\partial \mathbf{a}_{i-1}} =  \mathbf{W}_i^\intercal \delta_i$
>
> This becomes the new $\frac{\partial loss}{\partial \mathbf{a}_{i-1}}$ in the next backprop step, from which we can calculate $\delta_{i-1}$. 


<img src="https://www.cs.columbia.edu/~bauer/shape/nn_cg_backward.png" width=600px>

### Hints for Implementing Backpropagation

* For sigmoid activated single unit in the output (same as for the logistic regression case above):
  
  $\delta_L = \frac{\partial Loss}{\partial a_L} \frac{\partial a_L}{\partial z_L}  = \frac{y'-y}{y' (1-y')} \cdot y'  (1-y') = y'-y$  

* The same applies for softmax output and categorical cross-entropy loss! In other words $\delta_L$ is just the difference between the prediction and the target.

* It's not actually necessary to compute the $\frac{\partial loss}{\partial \mathbf{a}_i}$ terms explicitly. 
